# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [14]:
# импорт библиотек

import pandas as pd
import sklearn
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [15]:
# <чтение файла с данными с сохранением в data_full>

data = pd.read_csv('/datasets/insurance.csv')

In [16]:
# изучаем датафрейм

print(data.info())
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
None


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [17]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [18]:
data["Пол"].value_counts()

0    2505
1    2495
Name: Пол, dtype: int64

<b> Вывод: </b>

Данные состоят из 5000 строк

Признаки: Пол, Возраст, Зарплата, Члены, семьи

Целевой признак: количество страховых выплат

Данные в предобработке не нуждаются

Разделение данных по полу почти в равном колличестве

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

<b> Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.) </b>

**Ответ:** Не изменится.

**Обоснование:**

Признаки умножают на обратимую матрицу  :

$$
w1 = ((XP)^T XP)^{-1} (XP)^T y
$$

Транспонированное произведение матриц равно произведению транспонированных матриц, взятых в обратном порядке

$$
w1 = (P^T X^T XP)^{-1} P^T X^T y
$$

Используем формулу

$$
(A B)^{-1} = B^{-1} A^{-1}
$$

где A и B; $ P, P^T и X, X^T $ квадртаные 

$$
w1 = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$

Умножение матрицы на обратную матрицу равно единичной матрице

$$
w1 = P^{-1} (X^T X)^{-1} IX^T y
$$

$$
a1 = XPw1
$$

$$
a1 = XPP^{-1} (X^T X)^{-1} X^T y
$$

$$
a1 = X(X^T X)^{-1} X^T y
$$

$$
a1 = Xw
$$

<b> Вывод: </b>

Преобразовав формулы стало понятно, что при умножении признаков на обратимую матрицу качество не меняется.

## Алгоритм преобразования

**Алгоритм**

Необходимо умножить матрицу признаков на матрицу шифрования так как при умножении количество столбцов матрицы A должно быть равно количеству строк матрицы, полученная матрица будет иметь количество строк матрицы A и количество столбцов матрицы B. Поэтому размер матрицы щифрования на которую умножают равен 4х4

Матрицу шифрования сгенерируем с помощью np.random.normal()

**Обоснование**

In [19]:
#разобьем data на признаки и целевой признак
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']

In [20]:
# Создадим обратимую квадратную матрицу со случайными числами размерностью столбцов features
state = np.random.RandomState(12345)
matrix_cr=np.random.normal(size=(features.shape[1],features.shape[1]))
matrix_cr

array([[-1.51690857, -0.53460514, -0.21467252, -0.43861993],
       [-0.16897843, -0.9161244 ,  1.59661826, -0.97983047],
       [ 0.92525407,  0.67611277, -0.45815176,  0.08451254],
       [ 0.98430731,  1.02040552,  1.27933718,  0.79518379]])

In [27]:
# Проврка на обратимость созданной матрицы
matrix_cr_inv = np.linalg.inv(matrix_cr)
matrix_cr_inv_inv = np.linalg.inv(matrix_cr_inv)
matrix_cr_inv_inv

array([[-1.51690857, -0.53460514, -0.21467252, -0.43861993],
       [-0.16897843, -0.9161244 ,  1.59661826, -0.97983047],
       [ 0.92525407,  0.67611277, -0.45815176,  0.08451254],
       [ 0.98430731,  1.02040552,  1.27933718,  0.79518379]])

<b> Матрица обратима - при двойной инверсии получили исходную матрицу matrix_cr </b>

In [22]:
# зашифруем даннные путем умножения матриц
features_cr=features @ matrix_cr
features_cr

,0,1,2,3
0,45885.141307,33498.118276,-22657.801391,4152.005488
1,35152.866076,25651.164084,-17335.043186,3167.199494
2,19425.435160,14171.800640,-9574.885076,1746.348252
3,38581.514915,28176.704862,-19068.840825,3505.186837
4,24142.883003,17620.357305,-11913.270354,2177.903415
...,...,...,...,...
4995,33028.807627,24113.615349,-16308.753924,2991.252785
4996,48478.552470,35398.181518,-23951.587980,4395.938033
4997,31364.702123,22903.941352,-15496.853698,2846.968857
4998,30253.526677,22091.259575,-14942.813683,2743.950712


<b>Вывод</b>

После умножения размер матрицы признаков и кодированных признаков совпадает

## Проверка алгоритма

<b> Качество линейной регресии на нешифрованных данных </b>

In [23]:
model = LinearRegression()

model.fit(features, target)

predictions = model.predict(features)

print('R2:', r2_score(target,predictions))

R2: 0.4249455028666801


<b> Качество линейной регресии c шифрованием данных </b>

In [24]:
model = LinearRegression()

model.fit(features_cr, target)

predictions = model.predict(features_cr)

print('R2:', r2_score(target,predictions))

R2: 0.4249455028666794


<b>Вывод:</b>

Качество моделей с шифрованием при помощи умножения на обратимую матрицу со случайными числами не изменяется.

Небольшое расхождение предположу связанно с тем, что операции с float накапливают небольшую погрешность.